In [1]:
import pandas as pd
from pandas import read_csv
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import os

In [2]:
im_path = glob.glob('../../../images_training_rev1/*.jpg')

In [3]:
def crop_im(path_im):
    compteur = []
    
    for path in path_im:
        # récupération nom de l'image
        im_name = os.path.basename(path)

        # récupération des images
        img = cv2.imread(path)
        img = cv2.pyrDown(img)
        img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        

        # génération du masque de contour
        ret, thresh = cv2.threshold(img_grey, thresh=60, maxval=255, type=0)
        contours, hierarchy = cv2.findContours(thresh, 1, 2)

        # Récupération de la forme centrale
        middle_img = round(img.shape[0]/2)
        for contour in contours:
            if cv2.pointPolygonTest(contour,(middle_img,middle_img),False) == 1:
                cnt = contour
                continue

        # ciblage avec rectangle
        x,y,w,h = cv2.boundingRect(cnt)

        # récupération image inclue dans le rectangle
        img_crop = img[y:y+h, x:x+w]

        # sauvegarde nouvelle image
        new_folder = os.path.dirname(path) + '_resize/'
        cv2.imwrite(new_folder + im_name,img_crop)
        
        compteur.append({h,w}) 
        
    return compteur

In [4]:
c = crop_im(im_path)

In [5]:
import statistics
# Use: statistics.mean(liste)
 
from statistics import mean, stdev
# Use: mean(liste)

In [10]:
max_size = []
for compteur in c:
    max_size.append(max(compteur))
size = mean(max_size) + stdev(max_size)
size = round(size)
size

66

In [7]:
black = np.zeros((size,size,3), np.uint8)
cv2.imwrite('./blacksquare.jpg',black)

True

In [8]:
def fusion(s_img, black_img):
    x_offset= round((black_img.shape[1] - s_img.shape[1]) / 2 ) 
    y_offset= round((black_img.shape[0] - s_img.shape[0]) / 2 )
    black_img[y_offset:y_offset+s_img.shape[0], x_offset:x_offset+s_img.shape[1]] = s_img
    return black_img

In [17]:
im_path_resize = glob.glob('../../../images_training_rev1_resize/*.jpg')

In [18]:
def uniform(path_im_resize):
    for path in path_im_resize:
        # récupération nom de l'image
        im_name = os.path.basename(path)
        
        # récupération des images
        img = cv2.imread(path)

        # Mise à taille d'image identique
        black_img = cv2.imread("blacksquare.jpg")
        
        # Ignore les images trop grandes
        if img.shape[1] > black_img.shape[0] or img.shape[0] > black_img.shape[0]:
            continue
        
        # Mise à taille d'image identique
        img_black = fusion(img, black_img)
        
        # sauvegarde nouvelle image
        new_folder = os.path.dirname(path) + '_black/'
        cv2.imwrite(new_folder + im_name, img_black)
        
    return img_black.shape

In [19]:
sh = uniform(im_path_resize)
sh

(66, 66, 3)